<a href="https://colab.research.google.com/github/kavyabanka/Sabudh-AI-for-Book-Analysis/blob/main/Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163009 sha256=6dd2e8697261bee789eaa57acbbb6a1670e8212596a9ac92bb7c2b0f5bdf13bb
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
import pandas as pd
import surprise
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

data = pd.read_csv('/content/newgenre (1).csv')

reader = Reader(rating_scale=(1, 10000))
dataset = Dataset.load_from_df(data[['ISBN', 'Book-Title', 'Year-Of-Publication']], reader)
trainset, testset = train_test_split(dataset, test_size=0.2, random_state=42)
model = SVD()
model.fit(trainset)
predictions = model.test(testset)
accuracy.rmse(predictions)
def get_recommendations(isbn, model, data):
    all_isbns = data['ISBN'].unique()

    # Create a list of tuples with the ISBN and the predicted rating for the given user
    predictions = [(model.predict('ISBN', isbn).est, isbn) for isbn in all_isbns]

    recommendations = sorted(predictions, key=lambda x: x[0], reverse=True)

    return recommendations

isbn_to_recommend = '067003619X'
recommendations = get_recommendations(isbn_to_recommend, model, data)

top_n = 5
for i in range(top_n):
    print(f"Recommendation {i + 1}: ISBN {recommendations[i][1]}")


RMSE: 176.9969
Recommendation 1: ISBN 0684823802
Recommendation 2: ISBN 0345260317
Recommendation 3: ISBN 0671042858
Recommendation 4: ISBN 037582345X
Recommendation 5: ISBN 067103619X


In [ ]:
data.columns

Index(['Unnamed: 0', 'ISBN', 'Book-Title', 'Book-Author',
       'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M',
       'Image-URL-L', 'description', 'category'],
      dtype='object')

In [ ]:

data['Rating'] = 5.0
reader = Reader(rating_scale=(1, 10000))
dataset = Dataset.load_from_df(data[['ISBN', 'Book-Title', 'Rating']], reader)
trainset, testset = train_test_split(dataset, test_size=0.2, random_state=42)
model = SVD()
model.fit(trainset)
predictions = model.test(testset)
accuracy.rmse(predictions)

# Function to get book recommendations for a given ISBN
def get_recommendations(isbn, model, data):
    all_isbns = data['ISBN'].unique()

    # List of tuples with the ISBN and the predicted rating for the given user
    predictions = [(model.predict('ISBN', isbn).est, isbn) for isbn in all_isbns]

    # Sort the list by predicted rating in descending order
    recommendations = sorted(predictions, key=lambda x: x[0], reverse=True)

    return recommendations
isbn_to_recommend = '037582345X'
recommendations = get_recommendations(isbn_to_recommend, model, data)
top_n = 5
for i in range(top_n):
    print(f"Recommendatiocommendation {i + 1}: ISBN {recommendations[i][1]}")

RMSE: 0.0000
Recommendatiocommendation 1: ISBN 0684823802
Recommendatiocommendation 2: ISBN 0345260317
Recommendatiocommendation 3: ISBN 0671042858
Recommendatiocommendation 4: ISBN 037582345X
Recommendatiocommendation 5: ISBN 067103619X


In [ ]:
#cosine similarity - content

# Recommendation Based on Content

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

df = pd.read_csv("/content/newgenre (1).csv")


# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['description'])

In [ ]:
df = df.drop('Unnamed: 0',axis = 1)

In [ ]:
df['description'] = df['Book-Author'] + ' (' + df['Year-Of-Publication'].astype(str) + ') - ' + df['description']

In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix) #include additional computations

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix) #avoid unnecessary computations

In [ ]:
print(cosine_sim)

[[1.         0.07429304 0.04823276 ... 0.07037352 0.0239641  0.01986101]
 [0.07429304 1.         0.04866563 ... 0.07141415 0.04104521 0.02297302]
 [0.04823276 0.04866563 1.         ... 0.10567799 0.04182887 0.03054808]
 ...
 [0.07037352 0.07141415 0.10567799 ... 1.         0.05113842 0.02768493]
 [0.0239641  0.04104521 0.04182887 ... 0.05113842 1.         0.02300324]
 [0.01986101 0.02297302 0.03054808 ... 0.02768493 0.02300324 1.        ]]


In [ ]:
#NOISY DATA

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['description'])

In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
print(cosine_sim)

[[1.         0.02636206 0.01789997 ... 0.         0.         0.        ]
 [0.02636206 1.         0.         ... 0.         0.         0.        ]
 [0.01789997 0.         1.         ... 0.00661406 0.         0.        ]
 ...
 [0.         0.         0.00661406 ... 1.         0.00881114 0.        ]
 [0.         0.         0.         ... 0.00881114 1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [ ]:
# Function to get book recommendations based on the book title
def get_recommendations(book_title, cosine_sim=cosine_sim):
    idx = df[df['Book-Title'] == book_title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    book_indices = [i[0] for i in sim_scores]
    return df['Book-Title'].iloc[book_indices]

In [ ]:
df

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,description,category
0,0684823802,OUT OF THE SILENT PLANET,C.S. Lewis,1996,Scribner,http://images.amazon.com/images/P/0684823802.0...,http://images.amazon.com/images/P/0684823802.0...,http://images.amazon.com/images/P/0684823802.0...,C.S. Lewis (1996) - Dr. Ransom is abducted to ...,Fiction
1,0345260317,The Dragons of Eden: Speculations on the Evolu...,Carl Sagan,1978,Ballantine Books,http://images.amazon.com/images/P/0345260317.0...,http://images.amazon.com/images/P/0345260317.0...,http://images.amazon.com/images/P/0345260317.0...,Carl Sagan (1978) - Dr. Carl Sagan Takes Us on...,Behavior genetics
2,0671042858,The Girl Who Loved Tom Gordon,Stephen King,2000,Pocket,http://images.amazon.com/images/P/0671042858.0...,http://images.amazon.com/images/P/0671042858.0...,http://images.amazon.com/images/P/0671042858.0...,Stephen King (2000) - The acclaimed #1 New Yor...,Fiction
3,037582345X,"The Golden Compass (His Dark Materials, Book 1)",PHILIP PULLMAN,2002,Knopf Books for Young Readers,http://images.amazon.com/images/P/037582345X.0...,http://images.amazon.com/images/P/037582345X.0...,http://images.amazon.com/images/P/037582345X.0...,PHILIP PULLMAN (2002) - Living among scholars ...,Juvenile Fiction
4,067103619X,Lake News,Barbara Delinsky,2000,Pocket,http://images.amazon.com/images/P/067103619X.0...,http://images.amazon.com/images/P/067103619X.0...,http://images.amazon.com/images/P/067103619X.0...,Barbara Delinsky (2000) - Two people fleeing f...,American fiction
...,...,...,...,...,...,...,...,...,...,...
5675,0316909882,The further blurtings of Baxter,Glen Baxter,1994,"Little, Brown and Co",http://images.amazon.com/images/P/0316909882.0...,http://images.amazon.com/images/P/0316909882.0...,http://images.amazon.com/images/P/0316909882.0...,Glen Baxter (1994) - Another collection of biz...,Caricatures and cartoons
5676,0373123914,The Outback Marriage Ransom (Harlequin Presents),Emma Darcy,2004,Harlequin,http://images.amazon.com/images/P/0373123914.0...,http://images.amazon.com/images/P/0373123914.0...,http://images.amazon.com/images/P/0373123914.0...,Emma Darcy (2004) - Ric Donato wanted Lara Sey...,Abused women
5677,0751506451,The Maiden (The Morland Dynasty Series),Cynthia Harrod-Eagles,2001,Little Brown UK Ltd,http://images.amazon.com/images/P/0751506451.0...,http://images.amazon.com/images/P/0751506451.0...,http://images.amazon.com/images/P/0751506451.0...,Cynthia Harrod-Eagles (2001) - 1720: political...,Fiction
5678,0896382338,"My Child, My Teacher, My Friend: One Man's Vie...",Dwight Lee Wolter,1991,Hazelden,http://images.amazon.com/images/P/0896382338.0...,http://images.amazon.com/images/P/0896382338.0...,http://images.amazon.com/images/P/0896382338.0...,Dwight Lee Wolter (1991) - A collection of ess...,Social Science


In [ ]:
book_title = input()
recommendations = get_recommendations(book_title)
print(recommendations)

The further blurtings of Baxter
3083                Manifold: Time (Manifold (Hardcover))
4990                                                 Raft
2161      Police Procedurals (Academy Mysteries Novellas)
2602                                   Escape Not My Love
5329    Chicken Soup for the Kid's Soul (Mini Gift Books)
2431                      An Omelette and a Glass of Wine
1805                                       Orsinian Tales
735            Day In April  (Silhouette Big Summer Read)
4525                                         The Uprising
4902    The Legend of Sleepy Hollow and Other Stories:...
Name: Book-Title, dtype: object


# Recommendation Based on Fiction

In [ ]:
vectorizer = TfidfVectorizer(stop_words = 'english')
tfidf_matrix = vectorizer.fit_transform(df['category'])

In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
# Function to get book recommendations based on the book title
def get_recommendations(book_title, cosine_sim=cosine_sim):
    idx = df[df['Book-Title'] == book_title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    book_indices = [i[0] for i in sim_scores]
    return df['Book-Title'].iloc[book_indices]

In [ ]:
book_title = input()
recommendations = get_recommendations(book_title)
print(recommendations)

The further blurtings of Baxter
0                             OUT OF THE SILENT PLANET
1    The Dragons of Eden: Speculations on the Evolu...
2                        The Girl Who Loved Tom Gordon
3      The Golden Compass (His Dark Materials, Book 1)
4                                            Lake News
5                         The Adventures of Tom Sawyer
6                   Everything Is Illuminated: A Novel
7                                             Vanished
8                                      Almost: A Novel
9                     Chronique d'une mort annoncÃ?Â©e
Name: Book-Title, dtype: object


In [ ]:
df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,description,category
0,0684823802,OUT OF THE SILENT PLANET,C.S. Lewis,1996,Scribner,http://images.amazon.com/images/P/0684823802.0...,http://images.amazon.com/images/P/0684823802.0...,http://images.amazon.com/images/P/0684823802.0...,C.S. Lewis (1996) - Dr. Ransom is abducted to ...,Fiction
1,0345260317,The Dragons of Eden: Speculations on the Evolu...,Carl Sagan,1978,Ballantine Books,http://images.amazon.com/images/P/0345260317.0...,http://images.amazon.com/images/P/0345260317.0...,http://images.amazon.com/images/P/0345260317.0...,Carl Sagan (1978) - Dr. Carl Sagan Takes Us on...,Behavior genetics
2,0671042858,The Girl Who Loved Tom Gordon,Stephen King,2000,Pocket,http://images.amazon.com/images/P/0671042858.0...,http://images.amazon.com/images/P/0671042858.0...,http://images.amazon.com/images/P/0671042858.0...,Stephen King (2000) - The acclaimed #1 New Yor...,Fiction
3,037582345X,"The Golden Compass (His Dark Materials, Book 1)",PHILIP PULLMAN,2002,Knopf Books for Young Readers,http://images.amazon.com/images/P/037582345X.0...,http://images.amazon.com/images/P/037582345X.0...,http://images.amazon.com/images/P/037582345X.0...,PHILIP PULLMAN (2002) - Living among scholars ...,Juvenile Fiction
4,067103619X,Lake News,Barbara Delinsky,2000,Pocket,http://images.amazon.com/images/P/067103619X.0...,http://images.amazon.com/images/P/067103619X.0...,http://images.amazon.com/images/P/067103619X.0...,Barbara Delinsky (2000) - Two people fleeing f...,American fiction


In [ ]:
#Added the Author and Year of Publication in the description

In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 3.4 MB/s eta 0:00:00


# BERT

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

bert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

category_embeddings = bert_model.encode(df['category'].astype(str))
cosine_sim = cosine_similarity(category_embeddings, category_embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def get_recommendations(book_title, cosine_sim_matrix=cosine_sim, num_recommendations=10):
    idx = df[df['Book-Title'] == book_title].index[0]
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num_recommendations + 1]
    book_indices = [i[0] for i in sim_scores]
    return df['Book-Title'].iloc[book_indices]

book_title = "OUT OF THE SILENT PLANET"
recommendations = get_recommendations(book_title)
print(f"Recommendations for '{book_title}':")
print(recommendations)

Recommendations for 'OUT OF THE SILENT PLANET':
122                                          Burn Factor
123                                              Smitten
124                                     The Polished Hoe
125        The Last Heroes (Men at War (Paperback Jove))
126                                             Majestic
127    Welcome to the Great Mysterious (Ballantine Re...
135             Great Expectations (Wordsworth Classics)
136                                     The Manson Curse
139              Outlaw Mountain: A Joanna Brady Mystery
143           Castle of Wizardry (The Belgariad, Book 4)
Name: Book-Title, dtype: object


In [ ]:
category_embeddings = bert_model.encode(df['description'].astype(str))
cosine_sim = cosine_similarity(category_embeddings, category_embeddings)

In [ ]:
def get_recommendations(book_title, cosine_sim_matrix=cosine_sim, num_recommendations=10):
    idx = df[df['Book-Title'] == book_title].index[0]
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num_recommendations + 1]
    book_indices = [i[0] for i in sim_scores]
    return df['Book-Title'].iloc[book_indices]

book_title = "OUT OF THE SILENT PLANET"
recommendations = get_recommendations(book_title)
print(f"Recommendations for '{book_title}':")
print(recommendations)

Recommendations for 'OUT OF THE SILENT PLANET':
1391                 Storms of Victory  (Janissaries III)
2722                  Marooned (Star Trek Voyager, No 14)
2276                                   High Road to China
3214                                   Temptation's Trail
4711                          Doctor Who: Arc of Infinity
1899                                            Spellfall
56      Shadowrun: Find Your Own Truth (Secrets of Pow...
4285                                        Blind Pursuit
3164    The Door into Sunset (The Tale of the Five, Vo...
2998              Moonlight Rhapsody (Futuristic Romance)
Name: Book-Title, dtype: object


## For paraphrase identification

## These embeddings are numerical representations of the semantic content of the sentences according to the paraphrase-MiniLM-L6-v2 model

In [ ]:
#from transformers import AutoModel, BertTokenizerFast

In [ ]:
#bert = AutoModel.from_pretrained('bert-base-uncased')

# Load the BERT tokenizer
#tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')